# SET UP

In [ ]:
# Import requried libraries 
import pandas as pd
import numpy as np
import plotly.express as px
import time
from datetime import datetime
!pip install gspread
!pip install gspread_dataframe
from gspread_dataframe import set_with_dataframe
import gspread
gc = gspread.service_account(filename='/work/world-wide-web-356017-564c77672260.json')

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


# Extract

In [ ]:
# extract first df and create variable "prices". 
# list of default na values is redefined to exclude string 'NA' which happens to be Namibia's country code
prices = pd.read_csv('/work/cerebro/worldwide internet prices in 2022 - IN 2022.csv',keep_default_na=False, na_values=['-1.#IND', '1.#QNAN', '1.#IND', '-1.#QNAN', '#N/A N/A', '#N/A', 'N/A', 'n/a', '#NA', 'NULL', 'null', 'NaN', '-NaN', 'nan', '-nan', ''])

In [ ]:
# extract second df and create variable "speed"
speed = pd.read_csv('/work/cerebro/worldwide internet speed in 2022  - avg speed.csv')

In [ ]:
# extract third df and create variable "users"
users = pd.read_csv('/work/cerebro/worldwide internet users - users.csv')

# Transform

In [ ]:
# This is our DF
prices

,Country code,Name,Continental region,NO. OF Internet Plans,Average price of 1GB (USD),Cheapest 1GB for 30 days (USD),Most expensive 1GB (USD),Average price of 1GB (USD at the start of 2021),Average price of 1GB (USD – at start of 2020)
0,IL,Israel,NEAR EAST,27.0,$0.05,$0.02,$20.95,$0.11,$0.90
1,KG,Kyrgyzstan,CIS (FORMER USSR),20.0,$0.15,$0.10,$7.08,$0.21,$0.27
2,FJ,Fiji,OCEANIA,18.0,$0.19,$0.05,$0.85,$0.59,$3.57
3,IT,Italy,WESTERN EUROPE,29.0,$0.27,$0.09,$3.54,$0.43,$1.73
4,SD,Sudan,SUB-SAHARAN AFRICA,33.0,$0.27,$0.03,$0.92,$0.63,$0.68
...,...,...,...,...,...,...,...,...,...
237,VA,Vatican City (Holy See),Europe,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN
238,VE,Venezuela,SOUTH AMERICA,NaN,HYPERINFLATION,NaN,NaN,NaN,NaN
239,WF,Wallis and Futuna,OCEANIA,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN
240,CD,Congo (Democratic Republic of),SUB-SAHARAN AFRICA,NaN,Prices listed in non-convertible 'units',NaN,NaN,NaN,NaN


In [ ]:
# Checking prices columns, null values and data types
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 9 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Country code                                      242 non-null    object 
 1   Name                                              242 non-null    object 
 2   Continental region                                242 non-null    object 
 3   NO. OF Internet Plans                             230 non-null    float64
 4   Average price of 1GB (USD)                        242 non-null    object 
 5   Cheapest 1GB for 30 days (USD)                    230 non-null    object 
 6   Most expensive 1GB (USD)                          230 non-null    object 
 7   Average price of 1GB (USD  at the start of 2021)  230 non-null    object 
 8   Average price of 1GB (USD – at start of 2020)     230 non-null    object 
dtypes: float64(1), object(

#### DF has some few null values in 5 of the columns. Let's explore them.

In [ ]:
prices.loc[prices['NO. OF Internet Plans '].isnull(),]

,Country code,Name,Continental region,NO. OF Internet Plans,Average price of 1GB (USD),Cheapest 1GB for 30 days (USD),Most expensive 1GB (USD),Average price of 1GB (USD at the start of 2021),Average price of 1GB (USD – at start of 2020)
230,CW,Christmas Island,CARIBBEAN,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN
231,ER,Eritrea,NORTHERN AFRICA,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN
232,KP,North Korea,CARIBBEAN,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN
233,MH,Marshall Islands,OCEANIA,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN
234,PM,St. Pierre and Miquelon,CARIBBEAN,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN
235,SS,South Sudan,SUB-SAHARAN AFRICA,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN
236,TT,Tuvalu,OCEANIA,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN
237,VA,Vatican City (Holy See),Europe,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN
238,VE,Venezuela,SOUTH AMERICA,NaN,HYPERINFLATION,NaN,NaN,NaN,NaN
239,WF,Wallis and Futuna,OCEANIA,NaN,NO PROVIDERS,NaN,NaN,NaN,NaN


#### Indeed, this DF has basically no info from index 230 to 241, besides the country/region reference. We can drop this rows and we won't miss any valuable information.  

In [ ]:
# drop rows with null values for selected columns only
prices.dropna(subset=['NO. OF Internet Plans ','Average price of 1GB (USD)','Cheapest 1GB for 30 days (USD)',
    'Most expensive 1GB (USD)','Average price of 1GB (USD  at the start of 2021)',
    'Average price of 1GB (USD – at start of 2020)'],inplace=True)

#### Let's now check our DF in more detail, without null values

In [ ]:
prices

,Country code,Name,Continental region,NO. OF Internet Plans,Average price of 1GB (USD),Cheapest 1GB for 30 days (USD),Most expensive 1GB (USD),Average price of 1GB (USD at the start of 2021),Average price of 1GB (USD – at start of 2020)
0,IL,Israel,NEAR EAST,27.0,$0.05,$0.02,$20.95,$0.11,$0.90
1,KG,Kyrgyzstan,CIS (FORMER USSR),20.0,$0.15,$0.10,$7.08,$0.21,$0.27
2,FJ,Fiji,OCEANIA,18.0,$0.19,$0.05,$0.85,$0.59,$3.57
3,IT,Italy,WESTERN EUROPE,29.0,$0.27,$0.09,$3.54,$0.43,$1.73
4,SD,Sudan,SUB-SAHARAN AFRICA,33.0,$0.27,$0.03,$0.92,$0.63,$0.68
...,...,...,...,...,...,...,...,...,...
225,MW,Malawi,SUB-SAHARAN AFRICA,5.0,$25.46,$20.37,$25.46,$27.41,$3.59
226,ST,São Tomé and Príncipe,SUB-SAHARAN AFRICA,16.0,$30.97,$13.27,$121.65,$28.26,$5.33
227,SH,Saint Helena,SUB-SAHARAN AFRICA,4.0,$39.87,$38.57,$51.42,$52.50,$6.04
228,FK,Falkland Islands,SOUTH AMERICA,4.0,$44.56,$29.71,$82.26,$40.41,$55.47


#### Last five columns represent different types of internet costs for every country. We should convert those columns to integers and delete the $ sign for a better manipulation. 

In [ ]:
# First, let's apply a function to delete the $ sign 
cols = ['Average price of 1GB (USD)', 'Cheapest 1GB for 30 days (USD)', 'Most expensive 1GB (USD)','Average price of 1GB (USD  at the start of 2021)','Average price of 1GB (USD – at start of 2020)']
prices[cols] = prices[cols].apply(lambda x: x.str.lstrip('$'))

# now let's convert these columns to numeric
prices[cols] = prices[cols].apply(pd.to_numeric, errors= 'coerce')

# Finally, column names are a bit messy, we could improve them
prices.rename({'NO. OF Internet Plans ':'num_internet_plans','Average price of 1GB (USD)': 'avg_price_1gb','Cheapest 1GB for 30 days (USD)':'cheapest_1gb_30_days',
            'Most expensive 1GB (USD)':'most_expensive_1gb','Average price of 1GB (USD  at the start of 2021)':'avg_price_1gb_2021','Average price of 1GB (USD – at start of 2020)':'avg_price_1gb_2020'}, axis=1, inplace=True)

#### This should look better now.

In [ ]:
prices.columns

Index(['Country code', 'Name', 'Continental region', 'num_internet_plans',
       'avg_price_1gb', 'cheapest_1gb_30_days', 'most_expensive_1gb',
       'avg_price_1gb_2021', 'avg_price_1gb_2020'],
      dtype='object')

#### Once done with the numeric columns, let's check the ones containing text

In [ ]:
# As a good practice, let's get rid of whitespaces at the begining and end in all three columns
cols = ['Country code', 'Name', 'Continental region']
prices[cols] = prices[cols].apply(lambda x: x.str.strip())

# We can also improve the column names
prices.rename({'Country code':'country_code','Name': 'name','Continental region':'continental_region'}, axis=1, inplace=True)

#### Finally, let's check the "continental_region" column, it seems like with some transformations it can be insightful. 

In [ ]:
prices['continental_region'].value_counts()

SUB-SAHARAN AFRICA      47
WESTERN EUROPE          30
ASIA (EX. NEAR EAST)    28
CARIBBEAN               28
OCEANIA                 22
NEAR EAST               15
EASTERN EUROPE          14
SOUTH AMERICA           13
CIS (FORMER USSR)       11
CENTRAL AMERICA          8
NORTHERN AFRICA          7
NORTHERN AMERICA         4
BALTICS                  3
Name: continental_region, dtype: int64

#### Quite a few regions. Let's make it just 5: Asia, Americas, Europe, Africa and Oceania. I'll be usefull when visualizing this data in a map, to filter by region.

In [ ]:
# Reduce continental_region column to 5 regions
conditions = [ (prices['continental_region'].isin(('SOUTH AMERICA', 'CENTRAL AMERICA','NORTHERN AMERICA','CARIBBEAN'))),
                (prices['continental_region'].isin(('WESTERN EUROPE', 'EASTERN EUROPE', 'BALTICS'))),
                (prices['continental_region'].isin(('SUB-SAHARAN AFRICA', 'NORTHERN AFRICA'))),
                (prices['continental_region'].isin(('ASIA (EX. NEAR EAST)', 'NEAR EAST','CIS (FORMER USSR)'))),
                (prices['continental_region'].str.contains(('OCEANIA'))) ]
regions = ['Americas', 'Europe', 'Africa','Asia','Oceania']
prices["continental_region"] = np.select(conditions, regions, default=np.nan)

In [ ]:
# There seems to be a problem with a few european countries that figure as Asian. Let's correct this. 
prices.loc[prices['name'] == 'Belarus','continental_region'] = 'Europe'
prices.loc[prices['name'] == 'Ukraine','continental_region'] = 'Europe'
prices.loc[prices['name'] == 'Cyprus','continental_region'] = 'Europe'

#### It looks like we have a pretty clean data set now

In [ ]:
prices.describe()

,num_internet_plans,avg_price_1gb,cheapest_1gb_30_days,most_expensive_1gb,avg_price_1gb_2021,avg_price_1gb_2020
count,230.000000,230.000000,230.000000,230.000000,228.000000,227.000000
mean,26.730435,4.072957,1.722652,29.730348,5.091535,8.176960
std,16.468216,6.443007,4.774479,59.329737,6.653537,8.873163
min,1.000000,0.050000,0.000000,0.630000,0.090000,0.260000
25%,15.000000,1.082500,0.212500,5.902500,1.385000,2.850000
50%,22.000000,2.205000,0.630000,12.525000,3.240000,5.250000
75%,36.000000,4.357500,1.390000,34.295000,5.332500,11.035000
max,60.000000,49.670000,45.160000,768.870000,52.500000,65.830000


#### Let's now check the distributions of our numeric columns as they seem to have some outliers.

In [ ]:
fig = px.histogram(prices, x='avg_price_1gb', title='Distribution_avg_price_1gb', marginal='box')
fig.show()

In [ ]:
fig = px.histogram(prices, x='cheapest_1gb_30_days', title='Distribution_cheapest_1gb_30_days', marginal='box')
fig.show() 

In [ ]:
fig = px.histogram(prices, x='most_expensive_1gb', title='Distribution_most_expensive_1gb', marginal='box')
fig.show() 

In [ ]:
fig = px.histogram(prices, x='avg_price_1gb_2021', title='Distribution_avg_price_1gb_2021', marginal='box')
fig.show() 

In [ ]:
fig = px.histogram(prices, x='avg_price_1gb_2020', title='Distribution_avg_price_1gb_2020', marginal='box')
fig.show() 

#### As expected, these four distributions are heavily right skewed. Let's fix this to get good insights once we visualize the data.

In [ ]:
users

,Country or area,Subregion,Region,Internet users,Population
0,China,Eastern Asia,Asia,"1,010,740,000","1,427,647,786"
1,India,Southern Asia,Asia,"833,710,000","1,352,642,280"
2,United States,Northern America,Americas,"312,320,000","327,096,265"
3,Indonesia,South-eastern Asia,Asia,"196,000,000","267,670,543"
4,Brazil,South America,Americas,"160,010,801","209,469,323"
...,...,...,...,...,...
209,"Saint Helena, Ascension and Tristan da Cunha",Western Africa,Africa,"2,906","6,035"
210,Falkland Islands,South America,Americas,"2,881","3,234"
211,Montserrat,Caribbean,Americas,"2,833","4,993"
212,Wallis and Futuna,Polynesia,Oceania,"1,383","11,661"


In [ ]:
prices_speed_users = pd.merge(prices,speed,left_on='name',right_on='Country',how='inner')

In [ ]:
# prices_speed_users.loc[prices_speed_users['Country'].isnull(),]
prices_speed_users

,country_code,name,continental_region,num_internet_plans,avg_price_1gb,cheapest_1gb_30_days,most_expensive_1gb,avg_price_1gb_2021,avg_price_1gb_2020,Country,Avg \n(Mbit/s)Ookla
0,IL,Israel,Asia,27.0,0.05,0.02,20.95,0.11,0.90,Israel,28.01
1,KG,Kyrgyzstan,Asia,20.0,0.15,0.10,7.08,0.21,0.27,Kyrgyzstan,16.30
2,FJ,Fiji,Oceania,18.0,0.19,0.05,0.85,0.59,3.57,Fiji,25.99
3,IT,Italy,Europe,29.0,0.27,0.09,3.54,0.43,1.73,Italy,37.15
4,SD,Sudan,Africa,33.0,0.27,0.03,0.92,0.63,0.68,Sudan,9.50
...,...,...,...,...,...,...,...,...,...,...,...
128,GR,Greece,Europe,22.0,8.16,1.06,768.87,12.06,4.91,Greece,59.42
129,CY,Cyprus,Europe,22.0,8.84,2.06,59.10,13.56,20.25,Cyprus,75.48
130,YE,Yemen,Asia,17.0,15.98,0.40,39.94,15.98,15.73,Yemen,16.89
131,NA,Namibia,Africa,33.0,22.37,1.20,72.06,4.78,11.02,Namibia,18.43


In [ ]:
speed

,Country,Avg \n(Mbit/s)Ookla
0,Canada,75.18
1,South Korea,117.95
2,Netherlands,108.33
3,Japan,44.05
4,Norway,134.73
...,...,...
139,Sudan,9.50
140,Cuba,9.10
141,Tajikistan,7.05
142,Venezuela,5.68


# Load

In [ ]:
#load data to google sheets
#Create key
key = "1ta2iOwjKzIRu7CL60DqzXpVlYhvaTOz7618GmgF8Kts"

#Selects the sheet to export to
sheet = gc.open_by_key(key).worksheet('prices')
sheet1 = gc.open_by_key(key).worksheet('speed')
sheet2 = gc.open_by_key(key).worksheet('users')

#Clears the sheet and exports data
sheet.batch_clear(["A100:L40000"])
set_with_dataframe(sheet, prices) 

sheet1.batch_clear(["A100:R15000"])
set_with_dataframe(sheet1, speed) 

sheet2.batch_clear(["A100:Z15000"])
set_with_dataframe(sheet2, users)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=07a4ffe0-c8ed-4f4e-b576-dc2d51ba276d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>